In [1]:
import pandas as pd
import numpy as np
import geopy.distance
from models import *

In [5]:
# Find top 5 locations for a given location of a user

subquery = (Locations
            .select(Locations.street.alias('street'), fn.sqrt(fn.pow(51.69-Locations.dis_lat, 2) + 
                                                              fn.pow(5.30-Locations.dis_long, 2))
                    .alias('dist'), Locations.dis_long, Locations.dis_lat)
            .order_by(SQL('dist'))
            .limit(5)
            .alias('subquery'))

# pick one image for each of the top 5 locations

images = (Images
          .select(fn.min(Images.id).alias('id'), Images.street, Images.url, Images.title,
                  subquery.c.dis_long, subquery.c.dis_lat)
          .join(subquery, on =
                (Images.street == subquery.c.street))
          .group_by(Images.street)
          .order_by(subquery.c.dis_lat)
          .dicts()
          .execute())

pd.DataFrame(images)

,dis_lat,dis_long,id,street,title,url
0,51.68961,5.29943,745,Sint Jansstraat,"Zijgevel van het Keershuis, Lepelstraat 12.",http://denbosch.hosting.deventit.net/HttpHandl...
1,51.68973,5.30022,94,Korenbrugstraat,Binnendieze met beeld op hoek Korenbrugstraat/...,http://denbosch.hosting.deventit.net/HttpHandl...
2,51.68998,5.29982,430,Lepelstraat,Een schippersorkest loopt de Korenbrugstraat i...,http://denbosch.hosting.deventit.net/HttpHandl...
3,51.68998,5.29982,6411,Vismarkt,Markttafereel.,http://denbosch.hosting.deventit.net/HttpHandl...
4,51.69034,5.30036,108,Visstraat,Straatbeeld Visstraat te Den Bosch begin jaren...,http://denbosch.hosting.deventit.net/HttpHandl...


In [23]:
subquery = (Locations
            .select(Locations.street, fn.sqrt(fn.pow(51.69-Locations.latitude, 2) + 
                                              fn.pow(5.30-Locations.longitude, 2))
                    .alias('dist'), Locations.longitude, Locations.latitude,
                    fn.row_number().over(partition_by=[Locations.street], order_by=fn.Rand()),
                    Images.id)
            .join(Images, on=(Locations.street==Images.street))
            .dicts())
#             .order_by(SQL('dist'))
#             .alias('subquery'))

# pick one image for each of the top 5 locations

# images = (Images
#           .select(fn.min(Images.id).alias('id'), Images.street, Images.url, Images.title,
#                   subquery.c.dis_long, subquery.c.dis_lat)
#           .join(subquery, on =
#                 (Images.street == subquery.c.street))
#           .group_by(Images.street)
#           .order_by(subquery.c.dis_lat)
#           .dicts()
#           .execute())

pd.DataFrame(subquery).sort_values('street')

,`street` ORDER BY Rand()),dist,id,latitude,longitude,street
0,1,0.009771,30011,51.69252,5.29056,'s-Gravesandestraat
1,2,0.009771,39570,51.69252,5.29056,'s-Gravesandestraat
13,12,0.027665,26325,51.69811,5.32645,53rd Welsh Divisionplein
12,11,0.027665,40425,51.69811,5.32645,53rd Welsh Divisionplein
11,10,0.027665,31024,51.69811,5.32645,53rd Welsh Divisionplein
9,8,0.027665,11589,51.69811,5.32645,53rd Welsh Divisionplein
8,7,0.027665,3520,51.69811,5.32645,53rd Welsh Divisionplein
10,9,0.027665,10720,51.69811,5.32645,53rd Welsh Divisionplein
6,5,0.027665,39621,51.69811,5.32645,53rd Welsh Divisionplein
5,4,0.027665,32691,51.69811,5.32645,53rd Welsh Divisionplein


In [21]:
last_visited = (UserPictures
                .select(fn.max(UserPictures.picture_datetime))
                .where(UserPictures.user_id==12345))

long = (UserPictures
        .select(UserPictures.location_longitude)
        .where(UserPictures.user_id==12345 & UserPictures.picture_datetime==last_visited)
        .dicts())

lat = (UserPictures
        .select(UserPictures.location_latitude)
        .where(UserPictures.user_id==12345 & UserPictures.picture_datetime==last_visited)
        .dicts())

subquery_2 = (Locations
            .select(Locations.street, fn.sqrt(fn.pow(long-Locations.latitude, 2) + 
                                              fn.pow(lat-Locations.longitude, 2))
                    .alias('dist'), Locations.longitude, Locations.latitude)
            .order_by(SQL('dist'))
            .dicts()
            .limit(10)
            .alias('subquery_2'))

top_10_streets = list(pd.DataFrame(subquery_1)['street'])

subquery_3 = (UserPictures
             .select(UserPictures.image_id)
             .where(UserPictures.user_id==12345)
             .dicts())

subquery_4 = (Locations
            .select(fn.row_number().over(partition_by=[Locations.street], order_by=fn.Rand()).alias('rownr'),
                    Images.id, Images.street, Images.url, Images.title, Locations.longitude, Locations.latitude)
            .where(Images.street.in_(top_10_streets) & Images.usable==1 & Images.id.not_in(subquery_3))
            .join(Images, on=(Locations.street==Images.street))
            .dicts()
            .alias('subquery_4'))
              
suggestions = (Images
               .select()
               .join(subquery_4, on=(Images.id==subquery_4.c.id))
               .where(subquery_4.c.rownr==1)
               .order_by(subquery_4.c.latitude)
               .dicts()
               .execute())

pd.DataFrame(suggestions)

,id,street,title,url,usable,year
0,28537,Landstrekenlaan,Kinderen spelen met de ballonnen op de speelpl...,http://denbosch.hosting.deventit.net/HttpHandl...,1,1987
1,24158,Bollenveld,Portret van Truus van Mil en Albert Valentijn ...,http://denbosch.hosting.deventit.net/HttpHandl...,1,1990
2,25438,Eemland,Feestelijkheden bij de opening van de Merlijns...,http://denbosch.hosting.deventit.net/HttpHandl...,1,1987
3,41004,Binnenpolder,"Portret van Ron de Vries, marathonloper.",http://denbosch.hosting.deventit.net/HttpHandl...,1,1990
4,36811,Empelse Schans,In opdracht van Gemeentewerken 's-Hertogenbosc...,http://denbosch.hosting.deventit.net/HttpHandl...,1,1975
5,12083,Diepstraat,Gezien vanaf de Hekellaan. Panden die afgebrok...,http://denbosch.hosting.deventit.net/HttpHandl...,1,1953
6,2311,Sluisweg,Gebouwd in 1954. Opname in opdracht van Gemeen...,http://denbosch.hosting.deventit.net/HttpHandl...,1,1975
7,34776,Maasboulevard,Gezicht op de Maasbrug over de Maas in Maaspoort.,http://denbosch.hosting.deventit.net/HttpHandl...,1,2001
